
## 2. 模型总体框架

我们提出一个 **两级空间分层聚类 + 区域优先级评估** 的决策框架：

- **Level 1：空间热点识别**（州级风险分区）
- **Level 2：行动区域划分**（可执行调查单元）
- **Priority Scoring：行动顺序排序**

核心思想为：

> **政府的决策对象是“区域”，而非单条报告。**

---

## 3. 数据与坐标预处理

### 3.1 决策对象

仅考虑 Lab Status ∈ {`Unverified`, `Unprocessed`} 的报告作为 T3 决策输入。

### 3.2 坐标转换

为避免经纬度欧氏距离失真，将坐标近似转换为平面坐标（km）用于聚类；  
所有现实距离约束仍基于 Haversine 大圆距离。

---


## 4. Level 1：空间热点识别

### 4.1 方法



Silhouette 系数为：

$$
\boxed{
S(i)=\frac{b(i)-a(i)}{\max\{a(i),\,b(i)\}}
}
$$
来判断哪个k最好



## 5. Level 2：行动区域划分

设 Level 1 得到的某一空间热点区为 \(H\)，其内部包含若干公众报告。

若该热点区内任意两条报告之间的最大地理距离不超过政府单次行动所能覆盖的最大范围 \(D_{\max}\)，  
则该热点区可直接作为一个 **行动区域**。

若热点区内部空间跨度过大，则在该热点区内部进一步进行细分，  
采用 **凝聚型层次聚类（Agglomerative Clustering，Complete linkage）**，  
以控制每个子区域内的最大空间跨度。

细分后的每一个行动区域需同时满足以下现实约束：

- 区域内任意两点的最大地理距离不超过 \(D_{\max}\)，以保证单次行动可执行；
- 区域内包含不少于最小数量 \(n_{\min}\) 的报告，以避免信息不足的低效行动。

若在合理的细分范围内仍无法满足上述约束，则将该热点区整体标记为 **分散风险区**，  
并在后续优先级评估中整体处理。

---

## 6. 行动区域优先级评估

在完成 Level 2 行动区域划分后，设最终得到的行动区域集合为：

\[
\mathcal{R}=\{R_1, R_2, \dots, R_M\}
\]

政府需要在这些行动区域之间确定调查的优先顺序。

### 6.1 区域信息聚合原则（文字描述）

对每一个行动区域，综合考虑以下三类关键信息：

1. **风险强度（有效信息量）**  
   聚合区域内报告的预测正例概率，并限制单一区域内可贡献的有效信息上限，  
   以避免报告数量过多导致风险评估饱和。

2. **时间紧迫性（时间衰减）**  
   近期提交的报告被赋予更高权重，以反映胡蜂活动的时效性特征。

3. **行动可执行性（空间紧凑度）**  
   空间跨度越小、越紧凑的区域，其调查成本越低，行动效率越高。

---

### 6.2 综合优先级评分函数

综合上述因素，定义行动区域 \(R_j\) 的最终调查优先级为：

$$
\boxed{
\text{Priority}(R_j)
=
I(R_j)\cdot T(R_j)\cdot S(R_j)
}
$$

其中：

- \(I(R_j)\) 表示区域的有效风险信息量；
- \(T(R_j)\) 表示区域内报告的时间新近性；
- \(S(R_j)\) 表示区域在空间上的可执行性。

政府调查行动按照 Priority 值从高到低依次展开。

---




## 7. 总结

该模型通过 **空间分层聚类 + 区域级风险聚合**，  
将公众报告数据转化为 **可执行、可解释的政府调查决策**，  
在有限资源条件下实现调查效率与风险控制的平衡。


In [ ]:
import os
os.chdir(r"F:\2021C美赛")

import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score

# =========================
# 参数设置
# =========================

# T2 输出结果路径
T2_OUTPUT_PATH = r"data\\processed\\outputs_t2_ranked_reports.csv"

# T3 输出文件
OUTPUT_TABLE_PATH = r"data\\processed\\t3_region_priority_table.csv"
OUTPUT_FIG_PATH = r"data\\processed\\t3_region_clusters.png"

# --------- 关键建模参数 ---------

# Level 1：粗聚类（热点区）数量范围
LEVEL1_K_RANGE = range(2, 21)

# Level 2：政府一次行动可接受的最大直径（公里）
MAX_REGION_DIAMETER_KM = 400.0

# 区域优先级中“有效信息量”的上限（避免概率饱和）
K_EFFECTIVE = 10

# Level 2 最大允许的细分聚类数
MAX_K2 = 30


# =========================
# 地理工具函数
# =========================

def haversine_km(lat1, lon1, lat2, lon2):
    """
    使用 Haversine 公式计算地球表面两点之间的大圆距离（公里）
    """
    r = 6371.0
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    return 2 * r * np.arcsin(np.sqrt(a))


def max_intra_cluster_distance(coords):
    """
    计算同一聚类（区域）内任意两点的最大地理距离（公里）
    用于判断是否满足“单次政府行动可覆盖”的现实约束
    """
    if len(coords) < 2:
        return 0.0
    max_d = 0.0
    for i in range(len(coords)):
        for j in range(i + 1, len(coords)):
            d = haversine_km(
                coords[i, 0], coords[i, 1],
                coords[j, 0], coords[j, 1]
            )
            max_d = max(max_d, d)
    return max_d


def latlon_to_xy_km(coords):
    """
    将经纬度近似转换为平面坐标（单位：公里）
    用于聚类算法中避免直接使用经纬度欧氏距离的几何失真
    """
    lat = coords[:, 0]
    lon = coords[:, 1]

    lat0 = np.mean(lat)
    lon0 = np.mean(lon)

    x = (lon - lon0) * 111.32 * np.cos(np.radians(lat0))
    y = (lat - lat0) * 110.57
    return np.column_stack([y, x])


# =========================
# Step 1：读取并筛选数据
# =========================

df = pd.read_csv(T2_OUTPUT_PATH)

# 只保留未核实 / 未处理的报告（T3 决策对象）
df_t3 = df[df["Lab Status"].isin(["Unverified", "Unprocessed"])].copy()
df_t3 = df_t3.dropna(subset=["Latitude", "Longitude", "prob_positive"])

coords_latlon = df_t3[["Latitude", "Longitude"]].to_numpy()
coords_xy = latlon_to_xy_km(coords_latlon)

print(f"T3 reports count: {len(df_t3)}")

T3 reports count: 2352


# =========================
# Step 2：Level 1 粗聚类 —— 空间热点识别
# =========================

best_k1 = None
best_score = -1
best_labels_l1 = None

print("\nLevel 1 clustering (hotspot detection):")

for k in LEVEL1_K_RANGE:
    model = AgglomerativeClustering(n_clusters=k,linkage="ward")   #每次合并都让“类内平方和（SSE）增加得最少”
    labels = model.fit_predict(coords_xy)
    
    # ---------- (1) 极小簇过滤（防止 silhouette 虚高） ----------
    counts = np.bincount(labels)
    if counts.min() < 5:
        print(f"k={k}: skipped (min cluster size < 5)")
        continue
     # ---------- (2) 现实约束：最大可覆盖直径 ----------
    valid = True
    for c in np.unique(labels):
        coords_c = coords_latlon[labels == c]
        max_d = max_intra_cluster_distance(coords_c)
        if max_d > MAX_REGION_DIAMETER_KM:
            valid = False
            break
    if not valid:
        print(f"k={k}: skipped (region diameter > {MAX_REGION_DIAMETER_KM} km),(maxd={max_d})")
        continue

# ---------- (3) silhouette 稳定估计（多次采样取均值） ----------
    sample_size = min(1000, len(coords_xy))
    sils = []
    for seed in [0, 1, 2]:
        sils.append(
            silhouette_score(
                coords_xy,
                labels,
                sample_size=sample_size,
                random_state=seed
            )
        )
    sil = float(np.mean(sils))

    print(f"k={k}, silhouette={sil:.3f},(maxd={max_d})")
    # ---------- (4) 选择最优 k ----------
    if sil > best_score:
        best_score = sil
        best_k1 = k
        best_labels_l1 = labels
# ---------- (5) 写回结果 ----------
print(f"\nSelected Level-1 cluster number: {best_k1}")
print(f"Best silhouette score: {best_score:.3f}")

df_t3["cluster_l1"] = best_labels_l1


In [ ]:
# =========================
# Step 3：Level 2 细聚类 —— 行动区域划分（优化版）
# =========================

MIN_REPORTS_PER_REGION = 3

final_cluster_labels = np.full(len(df_t3), -1)
cluster_id = 0

print("\nLevel 2 clustering (actionable regions):")

for l1 in sorted(df_t3["cluster_l1"].unique()):
    idx = df_t3["cluster_l1"] == l1
    sub_coords_latlon = coords_latlon[idx]
    sub_coords_xy = coords_xy[idx]

    diameter = max_intra_cluster_distance(sub_coords_latlon)

    # ---------- (1) 若本身满足行动直径约束 ----------
    if diameter <= MAX_REGION_DIAMETER_KM:
        final_cluster_labels[idx.to_numpy()] = cluster_id
        print(f"Hotspot {l1}: single region (diameter {diameter:.1f} km OK)")
        cluster_id += 1
        continue

    # ---------- (2) 需要进一步细分 ----------
    found = False
    sub_idx = np.where(idx.to_numpy())[0]

    # k2 下界估计（现实导向）
    k2_min = max(2, int(np.ceil(diameter / MAX_REGION_DIAMETER_KM)))

    for k2 in range(k2_min, min(MAX_K2, len(sub_coords_xy))):
        model = AgglomerativeClustering(
            n_clusters=k2,
            linkage="complete"
        )
        labels_l2 = model.fit_predict(sub_coords_xy)

        ok = True
        for c in range(k2):
            mask = labels_l2 == c

            # 最小规模约束
            if np.sum(mask) < MIN_REPORTS_PER_REGION:
                ok = False
                break

            # 最大直径约束
            if max_intra_cluster_distance(sub_coords_latlon[mask]) > MAX_REGION_DIAMETER_KM:
                ok = False
                break

        if ok:
            print(f"Hotspot {l1}: split into {k2} regions")
            for c in range(k2):
                final_cluster_labels[sub_idx[labels_l2 == c]] = cluster_id
                cluster_id += 1
            found = True
            break

    # ---------- (3) fallback：高度分散区域 ----------
    if not found:
        final_cluster_labels[idx.to_numpy()] = cluster_id
        print(f"Hotspot {l1}: fallback as dispersed region (diameter {diameter:.1f} km)")
        cluster_id += 1

df_t3["cluster"] = final_cluster_labels
num_regions = df_t3["cluster"].nunique()

print(f"\nTotal actionable regions: {num_regions}")



Level 2 clustering (actionable regions):
Hotspot 0: single region (diameter 71.7 km OK)
Hotspot 1: single region (diameter 181.5 km OK)
Hotspot 2: single region (diameter 122.4 km OK)
Hotspot 3: single region (diameter 138.9 km OK)
Hotspot 4: single region (diameter 137.0 km OK)
Hotspot 5: single region (diameter 170.9 km OK)
Hotspot 6: single region (diameter 83.0 km OK)
Hotspot 7: single region (diameter 118.8 km OK)
Hotspot 8: single region (diameter 105.6 km OK)
Hotspot 9: single region (diameter 138.4 km OK)
Hotspot 10: single region (diameter 120.9 km OK)
Hotspot 11: single region (diameter 132.0 km OK)
Hotspot 12: single region (diameter 103.6 km OK)
Hotspot 13: single region (diameter 121.0 km OK)
Hotspot 14: single region (diameter 93.3 km OK)
Hotspot 15: single region (diameter 115.3 km OK)
Hotspot 16: single region (diameter 69.6 km OK)
Hotspot 17: single region (diameter 101.8 km OK)

Total actionable regions: 18


In [ ]:
# =========================
# Step 4：计算区域优先级（Top-K 有效概率）
# =========================

region_rows = []

for c in sorted(df_t3["cluster"].unique()):
    region_df = df_t3[df_t3["cluster"] == c]

    # 只取区域内概率最高的前 K 条报告，避免并集概率饱和
    probs = region_df["prob_positive"].to_numpy()
    top_probs = np.sort(probs)[-K_EFFECTIVE:] if len(probs) > K_EFFECTIVE else probs

    region_priority = 1.0 - np.prod(1.0 - top_probs)

    center_lat = region_df["Latitude"].mean()
    center_lon = region_df["Longitude"].mean()
    max_dist = max_intra_cluster_distance(
        region_df[["Latitude", "Longitude"]].to_numpy()
    )

    region_rows.append({
        "cluster": c,
        "center_latitude": center_lat,
        "center_longitude": center_lon,
        "num_reports": len(region_df),
        "priority": region_priority,
        "max_diameter_km": max_dist,
    })

region_table = pd.DataFrame(region_rows)
region_table = region_table.sort_values("priority", ascending=False)
region_table["priority_rank"] = np.arange(1, len(region_table) + 1)

# =========================
# Step 5：保存结果
# =========================

os.makedirs(os.path.dirname(OUTPUT_TABLE_PATH), exist_ok=True)
region_table.to_csv(OUTPUT_TABLE_PATH, index=False, encoding="utf-8")

print("\nTop regions by priority:")
print(region_table.head())
# =========================
# Step 6：空间可视化
# =========================

plt.figure(figsize=(8, 6))
plt.scatter(
    df_t3["Longitude"],
    df_t3["Latitude"],
    c=df_t3["cluster"],
    cmap="tab20",
    s=10,
    alpha=0.7
)

plt.scatter(
    region_table["center_longitude"],
    region_table["center_latitude"],
    c="black",
    marker="x",
    s=80,
    label="Region center"
)

plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("T3 Two-stage Spatial Clustering and Region Priority")
plt.legend()
plt.tight_layout()

plt.savefig(OUTPUT_FIG_PATH, dpi=300)
plt.close()

print(f"\nSaved region table to: {OUTPUT_TABLE_PATH}")
print(f"Saved clustering figure to: {OUTPUT_FIG_PATH}")


Top regions by priority:
    cluster  center_latitude  center_longitude  num_reports  priority  \
0         0        47.606180       -122.236268          639       1.0   
5         5        48.852185       -122.536351          307       1.0   
10       10        48.019860       -122.073553          186       1.0   
6         6        47.508789       -122.716641          167       1.0   
16       16        47.002042       -122.890598           92       1.0   

    max_diameter_km  priority_rank  
0         71.723008              1  
5        170.861495              2  
10       120.856928              3  
6         82.973767              4  
16        69.629198              5  

Saved region table to: data\\processed\\t3_region_priority_table.csv
Saved clustering figure to: data\\processed\\t3_region_clusters.png
